# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using surprise. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives
You will be able to:
* Fit a recommender system model to a set of data
* Create a function that will return the top recommendations for a user
* Introduce a new user to a rating matrix and make recommendations for them

For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last  lesson, you got exposed to working with Surprise datasets. In this lab, you will also go through the process of reading in a dataset into the Surprise dataset format. To begin with, load the dataset into a pandas dataframe. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [3]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [4]:
#drop unnecessary columns
new_df=df.drop(columns='timestamp')

It's now time to transform the dataset into something compatible with Surprise. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [5]:
!pip install scikit-surprise
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(new_df,reader)


    100% |████████████████████████████████| 3.3MB 8.3MB/s eta 0:00:01
    100% |████████████████████████████████| 286kB 29.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for scikit-surprise ... done
  Stored in directory: /home/amitrani/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [7]:
dataset = data.build_full_trainset()
print('Number of users: ',dataset.n_users,'\n')
print('Number of items: ',dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the Best Model
Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [8]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [9]:
## Perform a gridsearch with SVD
params = {'n_factors' :[20,50,100],
         'reg_all':[0.02,0.05,0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)


In [10]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.8690809225451691, 'mae': 0.6678751175808652}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 100, 'reg_all': 0.05}}


In [11]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)

In [12]:
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.97322292, 0.9685377 , 0.97069405, 0.97785083, 0.97294856]))
('test_mae', array([0.75209367, 0.7475225 , 0.75310834, 0.75377923, 0.75016285]))
('fit_time', (0.47995567321777344, 1.1271255016326904, 1.1780617237091064, 1.0096070766448975, 0.9952859878540039))
('test_time', (2.9990501403808594, 2.0245001316070557, 4.864514350891113, 4.381649971008301, 4.070740699768066))
-----------------------
0.9726508106642033


In [13]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [14]:
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0.87474912, 0.86800591, 0.87399948, 0.87638145, 0.87897161]))
('test_mae', array([0.66977236, 0.66250344, 0.66809988, 0.67026956, 0.67072806]))
('fit_time', (1.2921018600463867, 1.4459972381591797, 0.8556504249572754, 1.4792954921722412, 1.5554437637329102))
('test_time', (4.85075569152832, 5.022283554077148, 3.3077752590179443, 6.380891561508179, 6.0574541091918945))


0.8744215143234386

Based off these outputs, it seems like the best performing model is the SVD model with n_factors = 50 and a regularization rate of 0.05. Let's use that model to make some predictions. Use that model or if you found one that performs better, feel free to use that.

## Making Recommendations

This next section is going to involve making recommendations, and it's important that the output for the recommendation is interpretable to people. Rather than returning the movie_id values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peak at what information we have about them.

In [15]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [16]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [17]:

svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [18]:
svd.predict(2,4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0059784844645216, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put all of our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step to go let's create a function that allows students to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater` should take as parameters:
* movie_df : DataFrame - a dataframe containing the movie ids, name of movie, and genres
* num : int - number of ratings
* genre : string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int  , 'movieId': int  ,'rating': float  }

#### This function is optional, but fun :) 

In [21]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list
        

In [29]:
movie = df_movies.sample(8)
movie

,movieId,title,genres
1260,1675,Incognito (1997),Crime|Thriller
2664,3566,"Big Kahuna, The (2000)",Comedy|Drama
1155,1518,Breakdown (1997),Action|Thriller
1739,2334,"Siege, The (1998)",Action|Thriller
299,341,Double Happiness (1994),Drama
5991,36931,New Police Story (Xin jing cha gu shi) (2004),Action|Crime|Drama
6737,59141,Son of Rambow (2007),Children|Comedy|Drama
8918,135777,Golmaal (2006),Children|Comedy


In [30]:
user_rating = movie_rater(df_movies,4,'Comedy')

      movieId                 title                genres
4644     6942  Love Actually (2003)  Comedy|Drama|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
5
     movieId                title          genres
103      118  If Lucy Fell (1996)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId                        title           genres
5473    26198  Yours, Mine and Ours (1968)  Children|Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
5
      movieId            title        genres
8816   131013  Get Hard (2015)  Comedy|Crime
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4


user_rating = movie_rater(df_movies,4,'Comedy')

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [31]:
user_rating

[{'userId': 1000, 'movieId': 6942, 'rating': '5'},
 {'userId': 1000, 'movieId': 118, 'rating': '4'},
 {'userId': 1000, 'movieId': 26198, 'rating': '5'},
 {'userId': 1000, 'movieId': 131013, 'rating': '4'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a Surprise dataset
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number)

In [32]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [33]:
# train a model using the new combined DataFrame
svd_ = SVD(n_factors= 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [34]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))


In [35]:
list_of_movies

[(1, 4.168567968211908),
 (3, 3.6610691468512937),
 (6, 4.267378993849828),
 (47, 4.304998637086067),
 (50, 4.605847838039647),
 (70, 3.870679882188833),
 (101, 4.09699454067783),
 (110, 4.327045984356049),
 (151, 3.912279860485039),
 (157, 3.553408390390467),
 (163, 3.881141179045644),
 (216, 3.7201884036795896),
 (223, 4.246680210092519),
 (231, 3.3119442519465547),
 (235, 3.867604833872422),
 (260, 4.548347988233181),
 (296, 4.567287511214916),
 (316, 3.646860452051194),
 (333, 4.003020735964162),
 (349, 3.7960311179598802),
 (356, 4.421221665601084),
 (362, 3.8644890866487636),
 (367, 3.457815745690409),
 (423, 3.5701395113708463),
 (441, 4.21450436610195),
 (457, 4.320891117954714),
 (480, 4.096766978672579),
 (500, 3.766847926042798),
 (527, 4.514059493833419),
 (543, 3.7288540881493626),
 (552, 3.6407253087710445),
 (553, 4.174289390716848),
 (590, 3.993986526393816),
 (592, 3.7668310442037445),
 (593, 4.426914502021251),
 (596, 3.827020712348406),
 (608, 4.326648676071135),
 (6

In [36]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(list_of_movies,key=lambda x:x[1],reverse=True)

In [37]:
ranked_movies

[(1204, 4.726605061626877),
 (318, 4.68918054695306),
 (904, 4.685519484802576),
 (858, 4.64917365871264),
 (1104, 4.644090415379251),
 (2959, 4.639950312530002),
 (50, 4.605847838039647),
 (750, 4.592253806776212),
 (2324, 4.590533347096722),
 (3451, 4.580236357103192),
 (898, 4.571598471506278),
 (296, 4.567287511214916),
 (922, 4.565998032754703),
 (1197, 4.556678836914502),
 (741, 4.549488737640968),
 (260, 4.548347988233181),
 (56782, 4.547319503537083),
 (1228, 4.546544264133862),
 (3836, 4.545405061480615),
 (1208, 4.544664347027503),
 (1199, 4.543586112930225),
 (2329, 4.540492070314206),
 (1225, 4.536725812436994),
 (1272, 4.534010946139443),
 (1089, 4.5279734828055),
 (1136, 4.5193852232542975),
 (1196, 4.517819140859147),
 (48516, 4.515898423528262),
 (527, 4.514059493833419),
 (1276, 4.511774921978124),
 (2571, 4.508233672045837),
 (1237, 4.506475938467089),
 (1201, 4.5025691554038545),
 (57669, 4.501838524295805),
 (1215, 4.501725622944494),
 (866, 4.498989654977423),
 (12

 For the final component of this challenge, it could be useful to create a function `recommended_movies` that takes in the parameters:
* `user_ratings` : list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df` : DataFrame 
* `n` : int- number of recommended movies 

The function should use a for loop to print out each recommended *n* movies in order from best to worst

In [38]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']  
            
            print('Recommendation # ',idx+1,': ',title,'\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  906    Lawrence of Arabia (1962)
Name: title, dtype: object 

Recommendation #  2 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  3 :  686    Rear Window (1954)
Name: title, dtype: object 

Recommendation #  4 :  659    Godfather, The (1972)
Name: title, dtype: object 

Recommendation #  5 :  841    Streetcar Named Desire, A (1951)
Name: title, dtype: object 



## Level Up

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top n recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the change to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will get exposed to using spark to make recommender systems.